In [194]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
%config Completer.use_jedi = False
import SimpleITK as sitk
import nibabel as nib
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import logging
from utils_mask_sain import *
from utils import *
from utils_recaler import *
from utils_autour_3D import *


# Global: 3D

In [195]:
ls_image =sorted(glob.glob('./*_NAT*/*.nii'))

time_inj = ["_ART", "_PORT","_TARD"]
#ls_image_prime = ls_image[(82 + 242):]
ls_image_prime = ls_image
ls_image_no_time = natsorted(list(set([eliminate_temps(x) for x in ls_image_prime])))
ls_image_full_times_surconfiance = [[name +time + ".nii" for time in time_inj] for name in ls_image_no_time]
ls_image_full_time = []
dict_image_full_time = {}
for li_names in ls_image_full_times_surconfiance:
    li_true_names = []
    for i,name in enumerate(li_names):
        if name in ls_image_prime:
            li_true_names.append(name)
    ls_image_full_time.append(li_true_names)
    classe_name = li_names[0].split('/')[-2].split('_')[1]
    patient_num = li_names[0].split('/')[-1].split('_')[0]
    dict_image_full_time[(patient_num, classe_name)] = li_true_names

ls_image_full_time = ls_image_full_time

In [196]:
# # Alternative: use hardcoded settings (separate for settings, input image types and enabled features)
# settings = {}
# settings['binWidth'] = 25
# #settings['resampledPixelSpacing'] = [2,2]  # en 2D, on travaille dans un plan
# settings['resampledPixelSpacing'] = [2, 2, 2]  # This is an example for defining resampling (voxels with size 3x3x3mm)
# settings['interpolator'] = 'sitkBSpline'
# settings['verbose'] = True
# settings['force2D'] = True
# settings['force2Ddimension'] = 2


# extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
# extractor.settings
# median_spacing =  [0.59375, 0.59375, 1.5]

In [197]:
# label=extractor.settings.get('label')
# label=1
# extractor.disableAllFeatures()
# li_featureClass = ['firstorder','shape', 'glrlm', 'glszm', 'gldm', 'ngtdm']
# li_carac_glcm = ['Autocorrelation',
#     'ClusterProminence',
#     'ClusterShade',
#     'ClusterTendency',
#     'Contrast',
#     'Correlation',
#     'DifferenceAverage',
#     'DifferenceEntropy',
#     'DifferenceVariance',
#     'Id',
#     'Idm',
#     'Idmn',
#     'Idn',
#     'Imc1',
#     'Imc2',
#     'InverseVariance',
#     'JointEnergy',
#     'JointEntropy',
#     'MCC',
#     'MaximumProbability',
#     'SumAverage',
#     'SumEntropy',
#     'SumSquares'
# ]
# for featureClass in li_featureClass:
#     extractor.enableFeatureClassByName(featureClass)
# extractor.enableFeaturesByName(**{"glcm": li_carac_glcm})
# logger = logging.getLogger("radiomics.glcm")
# logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

In [198]:
# ### C est pas la bonne tumeur me semble t il au debut!!! Checker la bonne évolution: je crois qu'on extrait PAS les bons  trucs vs radio multislices
# all_dat = pd.DataFrame()
# do_nothing = False  ####### Si ZERO RESAMPLING
# no_decal = False    ####### SI ZERO DECALAGE
# show_plots = False
# for num, li_image_names in tqdm.tqdm(enumerate(ls_image_full_time)):
    
#     classe_name = li_image_names[0].split('/')[-2].split('_')[1]
#     patient_num = li_image_names[0].split('/')[-1].split('_')[0]
    
#     _ , _ , li_images, li_masks = equalize_slices(ls_image_full_time, dict_image_full_time,num = num, key = None, show = False, do_nothing = do_nothing, force_spacing = median_spacing)

#     for image_num_local, image in enumerate(li_images):
#         mask = li_masks[image_num_local]
#         imageName = li_image_names[image_num_local]
#         featureVector = extractor.execute(image, mask)
#         df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
#         df_idx['classe_name'] = classe_name
#         temps_inj = imageName.split('_')[-1].split('.')[0]
#         df_idx['temps_inj'] = temps_inj
#         df_idx['patient_num'] = patient_num
#         # al_dat_slice = al_dat_slice.append(df_idx)
#         df_idx = pd.DataFrame(df_idx).T
#         all_dat = pd.concat([df_idx,all_dat])
#         #print(al_dat_slice)


### vieux

In [199]:
# all_dat = pd.DataFrame()
# extractor.disableAllFeatures
# li_featureClass = ['firstorder', 'shape','glrlm', 'glszm', 'gldm', 'ngtdm']
# for featureClass in li_featureClass:
#     extractor.enableFeatureClassByName(featureClass)
# #On ne pourrait pas avoir les 2D...
# extractor.enableFeaturesByName(**{"glcm" : ['Autocorrelation',
#     'ClusterProminence',
#     'ClusterShade',
#     'ClusterTendency',
#     'Contrast',
#     'Correlation',
#     'DifferenceAverage',
#     'DifferenceEntropy',
#     'DifferenceVariance',
#     'Id',
#     'Idm',
#     'Idmn',
#     'Idn',
#     'Imc1',
#     'Imc2',
#     'InverseVariance',
#     'JointEnergy',
#     'JointEntropy',
#     'MCC',
#     'MaximumProbability',
#     'SumAverage',
#     'SumEntropy',
#     'SumSquares'
# ]})

# logger = logging.getLogger("radiomics.glcm")
# logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

# #OUILLE LES PATIENTS S'APPELLENT ./DATASET et pas en slices???

# for imageName in tqdm.tqdm(ls_image):
#     maskName = imageName.replace('_NAT','').replace('.nii','_masked.nii')
#     classe_name = imageName.split('/')[-2].split('_')[1]
#     temps_inj = imageName.split('_')[-1].split('.')[0]

#     patient_num = imageName.split('/')[-1].split('_')[0]
#     #print(patient_num) 

        
#     mask_sitk = sitk.ReadImage(maskName) !=  0  ### ATTENTION C'est important ici on s'interesse au masque et non a l'image masquée
#     # print(imageName)
#     # print(mask_sitk.GetSize())
#     #print(maskName)
#     mr_sitk = sitk.ReadImage(imageName)
#     #print("nbe voxels :")
#     #print(sitk.GetArrayFromImage(mask_sitk).sum())


#     featureVector = extractor.execute(mr_sitk, mask_sitk)
#     df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
#     df_idx['classe_name'] = classe_name
#     df_idx['temps_inj'] = temps_inj
#     df_idx['patient_num'] = patient_num
#     df_idx = pd.DataFrame(df_idx).T
#     all_dat = pd.concat([df_idx,all_dat])
    


In [200]:
# all_dat.to_csv('./liver_tumors_all_LLB.csv')
# all_dat.to_excel('./global_excel.xlsx', index = False)

# Multislice

### Sans l'extraction de partie saine

In [201]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = [2,2]
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 2


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 2,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

DEGAGER

In [202]:
# time_inj = ["_ART", "_PORT", "_VEIN","_TARD"]
# # ls_image_prime = ls_image[(82 + 242):]
# ls_image_prime = ls_image
# ls_image_no_time = natsorted(list(set([eliminate_temps(x) for x in ls_image_prime])))
# ls_image_full_times_surconfiance = [[name +time + ".nii" for time in time_inj] for name in ls_image_no_time]
# ls_image_full_time = []
# dict_image_full_time = {}
# for li_names in ls_image_full_times_surconfiance:
#     li_true_names = []
#     for i,name in enumerate(li_names):
#         if name in ls_image_prime:
#             li_true_names.append(name)
#     ls_image_full_time.append(li_true_names)
#     classe_name = li_names[0].split('/')[-2].split('_')[1]
#     patient_num = li_names[0].split('/')[-1].split('_')[0]
#     dict_image_full_time[(patient_num, classe_name)] = li_true_names
# # ls_image_full_time = ls_image_full_time[0:2]
# # print(ls_image_full_time[0])

FIN DEGAGER

In [203]:
# li_spacing = []
# for num, names in enumerate(ls_image_full_time):
#     for i_time, name in enumerate(names):
#         maskName = name.replace('_NAT','').replace('.nii','_masked.nii')
#         classe_name = name.split('/')[-2].split('_')[1]
#         temps_inj = name.split('_')[-1].split('.')[0]
#         patient_num = name.split('/')[-1].split('_')[0]
#         mask_sitk = sitk.ReadImage(maskName) !=  0
#         image = sitk.ReadImage(name)
#         li_spacing.append(list(image.GetSpacing()))
# li_spacing = np.array(li_spacing)
# print(li_spacing.shape)
# median_spacing = np.median(li_spacing, axis = 0)/2
# print(np.std(li_spacing, axis = 0))
median_spacing =  [0.59375, 0.59375, 1.5]
        

In [204]:
mode = "area"
fenetre_x = None
fenetre_y = None
fenetre_z  = range(-10,10)
label=extractor.settings.get('label')
label=1
minDims=2
extractor.disableAllFeatures()
#utiliser des try et excepts ...
li_featureClass = ['firstorder','shape2D', 'glrlm', 'glszm', 'gldm', 'ngtdm']
li_carac_glcm = ['Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    'SumAverage',
    'SumEntropy',
    'SumSquares'
]
for featureClass in li_featureClass:
    extractor.enableFeatureClassByName(featureClass)
extractor.enableFeaturesByName(**{"glcm": li_carac_glcm})
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)
al_dat_slice = pd.DataFrame()
# path_slice = "../multislice_excel_with_shape_2D_10_min.xlsx"
# al_dat_slice = pd.read_excel(path_slice, engine='openpyxl')

utiliser la pipeline de feature extraction individuellement sur chaque tumeur: exit le recalage

In [205]:
do_nothing = False  ####### Si ZERO RESAMPLING
no_decal = False    ####### SI ZERO DECALAGE
show_plots = False
n_slices = 10
for num, li_image_names in tqdm.tqdm(enumerate(ls_image_full_time)):
    #li_mask_names = [image_name.replace('_NAT','').replace('.nii','_masked.nii') for image_name in li_image_names]
    
    
    classe_name = li_image_names[0].split('/')[-2].split('_')[1]
    patient_num = li_image_names[0].split('/')[-1].split('_')[0]
    
    # li_images = [sitk.ReadImage(image_name) for image_name in li_image_names]
    # li_mask = [sitk.ReadImage(mask_name) !=  0 for mask_name in li_mask_names]
    sizes = [np.array(sitk.ReadImage(image_name).GetSize())[2] for image_name in li_image_names]
    max_size = np.max(sizes)
    difference = max_size - sizes
    
    li_images = [sitk.ReadImage(image_name) for image_name in li_image_names]
    li_masks = [sitk.ReadImage(image_name.replace('_NAT','').replace('.nii','_masked.nii')) !=  0 for image_name in li_image_names]
    
    
    #même échelle selon x et y pour tout le monde.
    li_num_slices = np.array([image.GetSize()[2] for image in li_images])
    image_num_with_max_slices = np.argmax(li_num_slices)
    for i, image in enumerate(li_images):
        resampled_image, resampler = resample_image_to_reference(image, image,median_spacing[:2]) 
        resampled_mask = resampler.Execute(li_masks[i]) != 0
        li_images[i] = resampled_image
        li_masks[i] = resampled_mask


    li_shapes = [list(image.GetSize()) for image in li_images]
    li_shapes_masks = [list(mask.GetSize()) for mask in li_masks]
    # print("shapes", li_shapes)
    # print("shapes masks", li_shapes_masks)
    #Calculer la nouvelle échelle selon z 
    li_spacings_z, li_new_num_slices, li_heights_true_size = find_new_scale(li_images, li_masks,n_slices, median_spacing)
    li_spacings_z = np.array(li_spacings_z)
    li_heights_true_size = np.array(li_heights_true_size)
    new_num_slices = max(li_new_num_slices) #assurer même nombre de slices pour chacun
    li_images_attemp = []
    li_masks_attemp =  []
    n_slices_local = n_slices
    too_short = False #s'active si on a déjà trop réduit une image une fois
    n_refus = 0 #compte le nombre de refus
    while len(li_images_attemp) < len(li_images):
        i = len(li_images_attemp)
        image = li_images[i]
        new_shape = list(image.GetSize())
        new_shape[2] = new_num_slices
        #new_image, resampler = resample_image_to_reference(image, image, [median_spacing[0], median_spacing[1], li_spacings_z[i]], force_size = new_shape)
        new_image, resampler = resample_image_to_reference(image, image, [median_spacing[0], median_spacing[1], np.min(li_spacings_z)])
        new_mask = resampler.Execute(li_masks[i]) != 0
        li_slices_image_local  = [new_image[:,:,z] for z in range(new_image.GetSize()[2])]
        li_slices_mask_local = [new_mask[:,:,z] for z in range(new_mask.GetSize()[2])]
        num_size_good = len(get_good_slices_from_li(li_slices_image_local, li_slices_mask_local))
        acceptable = True #ces nombres de slices sont-ils acceptables?
        if num_size_good < n_slices:
            #print(f"C'est trop petit! Ouille! num_size_good = {num_size_good} pour i = {i}, n_slices_local = {n_slices_local} and min_li_spacing = {li_spacings_z}")
            n_refus += 1
            li_images_attemp = []
            li_masks_attemp = []
            n_slices_local = n_slices_local + 0.05
            li_spacings_z = li_heights_true_size/n_slices_local
            too_short = True # ne plus essayer de réduire désormais: ça rendrait trop petit
        elif i == np.argmin(li_spacings_z) and num_size_good > n_slices and not too_short:
            #print("on réduit", li_spacings_z)
            n_refus += 1
            #print("inacceptable: ", i, num_size_good, f" n_slices_local = {n_slices_local} and min_li_spacing = {np.min(li_spacings_z)}")
            li_images_attemp = []
            li_masks_attemp = []
            n_slices_local = n_slices_local - 0.1
            li_spacings_z  = li_heights_true_size/n_slices_local
        else:    
            li_images_attemp.append(new_image)
            li_masks_attemp.append(new_mask)
        
    li_images = li_images_attemp
    li_masks = li_masks_attemp
    
    len_good_images = [len(get_good_slices_from_li([image[:,:,z] for z in range(image.GetSize()[2])], [mask[:,:,z] for z in range(mask.GetSize()[2])])) for image, mask in zip(li_images, li_masks)]
    print("len good images", len_good_images)
    if np.min(len_good_images) != n_slices:
        raise ValueError("pas le bon nombre de slices")
    ref_for_decal = np.argmin(np.array(len_good_images))
            
            
    li_slices_images = [[image[:,:,z] for z in range(image.GetSize()[2])] for image in li_images]
    li_slices_masks = [[mask[:,:,z] for z in range(mask.GetSize()[2])] for mask in li_masks]
    
    # li_slices_masks, li_slices_images, _, _ = equalize_slices(ls_image_full_time, dict_image_full_time,num = num, key = None, show = False, do_nothing = do_nothing, force_spacing = median_spacing)
    if no_decal:
        li_decal = np.zeros_like(li_decal)
    else:
        _,_,li_decal = find_best_all_area(li_masks, fenetre_z, fenetre_x, fenetre_y, ref_num = ref_for_decal)
        li_decal = np.array(li_decal)
        print("decalage", li_decal)
    li_caled = []
    #liste de l'individu de la key d'intérêt
    for image_num_local, li_slices_image_unique in enumerate(li_slices_images):
        nb_slices = len(li_slices_images[image_num_local])
        li_caled.append({}) #une liste par radio
        li_slices_mask_unique = li_slices_masks[image_num_local]
        slice_depart = abs(min(li_decal[image_num_local],0))
        slice_fin = nb_slices - np.max(li_decal[image_num_local],0)
        #print("num", num, "num_time", image_num_local,"slice_depart", slice_depart, "slice_fin", slice_fin)
        for slice_num_ref in range(slice_depart, slice_fin):
            slice_num = slice_num_ref + li_decal[image_num_local]
            slice_mask = li_slices_mask_unique[slice_num]   #format sitk
            slice_img = li_slices_image_unique[slice_num]   #format sitk
            lsif = sitk.LabelStatisticsImageFilter()
            lsif.Execute(slice_img, slice_mask)
            boundingBox = np.array(lsif.GetBoundingBox(label))
            ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
            li_caled[-1][slice_num_ref] = {"img": slice_img, "mask": slice_mask, "recal": li_decal[image_num_local], "accept": False, "diff": difference[image_num_local], "time": time_inj[image_num_local]}
            if (sitk.GetArrayFromImage(slice_mask).sum() > 0) & (ndims >= minDims):
                try:
                    li_caled[-1][slice_num_ref]["accept"] = True #Comprendre qui est accepté
                    imageName = li_image_names[image_num_local]
                    featureVector = extractor.execute(slice_img, slice_mask)
                    featureVector['slice_num'] = slice_num_ref #Tres important: on ne veut pas le slice_num mais le slice_num_ref
                    df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
                    df_idx['classe_name'] = classe_name
                    temps_inj = imageName.split('_')[-1].split('.')[0]
                    df_idx['temps_inj'] = temps_inj
                    df_idx['patient_num'] = patient_num
                    # al_dat_slice = al_dat_slice.append(df_idx)
                    df_idx = pd.DataFrame(df_idx).T
                    al_dat_slice = pd.concat([al_dat_slice, df_idx])
                    #print(al_dat_slice)
                except Exception as e:
                    print("problem with slice", slice_num, "of image", imageName, "erreur", e)
    if show_plots:
        set_slice_num_ref_present = set([])
        for radio in li_caled:
            set_slice_num_ref_present = set(list(radio.keys())).union(set_slice_num_ref_present)
        slice_num_ref_present = list(set_slice_num_ref_present)
        for num_ref in slice_num_ref_present:
            li_slices = []
            for num_radio in range(len(li_caled)):
                if num_ref in list(li_caled[num_radio].keys()):
                    li_slices.append(li_caled[num_radio][num_ref])
            plt.figure(figsize = (15,5))
            for num_radio, dic_slice in enumerate(li_slices):
                mask_superpose_simple(dic_slice["img"], dic_slice["mask"], num = num_radio + 1, max_num = len(li_slices), legend = f"patient ls num = {num} slice {num_ref}, recalage = {dic_slice['recal']},\n accept = {dic_slice['accept']}, diff = {dic_slice['diff']}, time = {dic_slice['time']}")
            plt.subplots_adjust(wspace=2.5)
            plt.show()
        


0it [00:03, ?it/s]


KeyboardInterrupt: 

In [97]:
al_dat_slice.to_csv('./liver_tumors_slices_LLB.csv')
al_dat_slice.to_excel('../multislice_excel_with_shape_2D_10_min.xlsx', index = False)

### Avec extraction de partie saine

In [166]:
# settings = {}
# settings['binWidth'] = 25
# settings['resampledPixelSpacing'] = [2,2]
# #settings['resampledPixelSpacing'] = [2, 2, 2]  # This is an example for defining resampling (voxels with size 3x3x3mm)
# settings['interpolator'] = 'sitkBSpline'
# settings['verbose'] = True
# settings['force2D'] = True
# settings['force2Ddimension'] = 2   # ne pas oublier de le changer : fait: ce n'est plus True

# settings_sain = {}
# settings_sain['binWidth'] = 25
# settings_sain['resampledPixelSpacing'] = [2,2,2]
# interpolator = 'sitkBSpline'
# settings_sain['interpolator'] = interpolator
# settings_sain['verbose'] = True
# settings_sain['force2D'] = True
# settings_sain['force2Ddimension'] = 2



# extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
# extractor_sain = featureextractor.RadiomicsFeatureExtractor(**settings_sain)

In [167]:
# li_spacing = []
# for num, names in enumerate(ls_image_full_time):
#     for i_time, name in enumerate(names):
#         maskName = name.replace('_NAT','').replace('.nii','_masked.nii')
#         classe_name = name.split('/')[-2].split('_')[1]
#         temps_inj = name.split('_')[-1].split('.')[0]
#         patient_num = name.split('/')[-1].split('_')[0]
#         mask_sitk = sitk.ReadImage(maskName) !=  0
#         image = sitk.ReadImage(name)
#         li_spacing.append(list(image.GetSpacing()))
# li_spacing = np.array(li_spacing)
# print(li_spacing.shape)
# median_spacing = np.median(li_spacing, axis = 0)/2
# print(np.std(li_spacing, axis = 0))
# print(median_spacing)
median_spacing =  [0.59375, 0.59375, 1.5]

In [168]:
# #mode = "area"
# fenetre_z = range(-10, 11)
# fenetre_x = range(-4, 5)
# fenetre_y = range(-4, 5)
# label=extractor.settings.get('label')
# label=1
# minDims=2
# extractor.disableAllFeatures()
# extractor_sain.disableAllFeatures()
# #utiliser des try et excepts ...
# li_featureClass = ['firstorder','shape2D', 'glrlm', 'glszm', 'gldm', 'ngtdm']
# li_featureClass_sain = ['firstorder', 'glrlm', 'glszm', 'gldm', 'ngtdm']
# li_carac_glcm = ['Autocorrelation',
#     'ClusterProminence',
#     'ClusterShade',
#     'ClusterTendency',
#     'Contrast',
#     'Correlation',
#     'DifferenceAverage',
#     'DifferenceEntropy',
#     'DifferenceVariance',
#     'Id',
#     'Idm',
#     'Idmn',
#     'Idn',
#     'Imc1',
#     'Imc2',
#     'InverseVariance',
#     'JointEnergy',
#     'JointEntropy',
#     'MCC',
#     'MaximumProbability',
#     'SumAverage',
#     'SumEntropy',
#     'SumSquares'
# ]
# for featureClass in li_featureClass:
#     extractor.enableFeatureClassByName(featureClass)
    new_num_slices = max(li_new_num_slices) #assurer même nombre de slices pour chacun
# for featureClass in li_featureClass_sain:
#     extractor_sain.enableFeatureClassByName(featureClass)
# extractor.enableFeaturesByName(**{"glcm": li_carac_glcm})
# extractor_sain.enableFeaturesByName(**{"glcm": li_carac_glcm})
# logger = logging.getLogger("radiomics.glcm")
# logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)
# al_dat_slice = pd.DataFrame()
# data_sain = pd.DataFrame()
# # path_sain = '../liver_sain.xlsx'
# # data_sain = pd.read_excel(path_sain, engine='openpyxl')
# # print(data_sain.dtypes)

IndentationError: unexpected indent (1706139861.py, line 39)

In [149]:
# do_nothing = False  ####### Si ZERO RESAMPLING
# no_decal = False    ####### SI ZERO DECALAGE
# show_plots = False
# time_ref = 3
# double = True #extraire aussi multislice?? Si non: respecter le temps_ref déjà utilisé...


# for num, li_image_names in tqdm.tqdm(enumerate(ls_image_full_time)):
#     #li_mask_names = [image_name.replace('_NAT','').replace('.nii','_masked.nii') for image_name in li_image_names]
    
#     classe_name = li_image_names[0].split('/')[-2].split('_')[1]
#     patient_num = li_image_names[0].split('/')[-1].split('_')[0]
    
#     # li_images = [sitk.ReadImage(image_name) for image_name in li_image_names]
#     # li_mask = [sitk.ReadImage(mask_name) !=  0 for mask_name in li_mask_names]
#     sizes = [np.array(sitk.ReadImage(image_name).GetSize())[2] for image_name in li_image_names]
#     max_size = np.max(sizes)
#     difference = max_size - sizes
    
#     li_slices_masks, li_slices_images,li_images, li_masks = equalize_slices(ls_image_full_time, dict_image_full_time,num = num, key = None, show = False, do_nothing = do_nothing, force_spacing = median_spacing)
#     li_enum = list(range(len(li_slices_images)))
#     if len(li_enum) == len(time_inj):
#         li_enum[time_ref] = 0
#         li_enum[0] = time_ref
#         local_time_ref = time_ref
#     else: 
#         local_time_ref = 0
#     if no_decal:
#         li_decal = np.zeros_like(li_decal)
#     else:
#         li_best_shift_x, li_best_shift_y, li_decal = find_best_all_area(li_masks, fenetre_z, fenetre_x, fenetre_y, local_time_ref)
#         li_best_shift_z = [li_masks[0].GetSpacing()[2]*decal for decal in li_decal]
#     li_caled = {}
#     #liste de l'individu de la key d'intérêt
#     for i_enum in range(len(li_slices_images)):
#         image_num_local = li_enum[i_enum]
#         li_slices_image_unique = li_slices_images[image_num_local]
#         nb_slices = len(li_slices_images[image_num_local])
#         li_caled[image_num_local] = {} #un dico par radio
#         li_slices_mask_unique = li_slices_masks[image_num_local]
#         slice_depart = abs(min(np.min(li_decal[image_num_local]),0))
#         slice_fin = nb_slices - max(np.max(li_decal[image_num_local]),0)
#         # print("fin", slice_depart, slice_fin, nb_slices, li_decal)
#         image = li_images[image_num_local]
#         mask = li_masks[image_num_local]
#         imageName = li_image_names[image_num_local]
#         temps_inj = imageName.split('_')[-1].split('.')[0]
        
#         if image_num_local == local_time_ref:
#             echec = True
#             erosion = 5
#             while echec:
#                 mask_tot_ref = pipeline_3D(image, mask, sigma = 1, quantile = 0.4, interieur = 8, exterieur = 45, erosion = erosion, show = False, slice = 110)
#                 good_slices = get_good_slices_from_li(li_slices_images[local_time_ref], [mask_tot_ref[:,:,z] for z in range(mask_tot_ref.GetSize()[2])])
#                 good_slices_tumor = get_good_slices_from_li(li_slices_images[local_time_ref], [mask[:,:,z] for z in range(mask.GetSize()[2])])
#                 if len(good_slices) > 0:
#                     echec = False
#                 else:
#                     erosion -= 1
#                     print(f"Nouvelle erosion = {erosion}")
#             mask_sain = sitk.Image(mask_tot_ref)

#         else:
#             mask_sain = translate_mask(mask_tot_ref, li_best_shift_x[image_num_local], li_best_shift_y[image_num_local], li_best_shift_z[image_num_local])
            
#         #Extraire pour masque sain
#         try:
#             featureVectorSain = extractor_sain.execute(image, mask_sain)
#             df_idx_sain = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVectorSain.items() ])).iloc[0]
#         except Exception as e:
#             print("pb with mask sain of image", imageName, "erreur", e)
#             featureVectorSain = extractor_sain.execute(image, mask)
#             df_idx_sain = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVectorSain.items() ])).iloc[0]
#             df_idx_sain[:] = np.nan
#         df_idx_sain['temps_inj'] = temps_inj
#         df_idx_sain['patient_num'] = patient_num
#         df_idx_sain['classe_name'] = classe_name
#         df_idx_sain = pd.DataFrame(df_idx_sain).T
#         data_sain = pd.concat([data_sain,df_idx_sain])
            
#         if double:
#             for slice_num_ref in range(slice_depart, slice_fin):
#                 # print(mask_sain.GetSize())
#                 # print(mask.GetSize())
#                 # print(image.GetSize())
#                 # print(nb_slices)
#                 # print(slice_fin)
#                 # print(slice_num_ref)
#                 slice_num = slice_num_ref + li_decal[image_num_local]
#                 slice_mask = li_slices_mask_unique[slice_num]   #format sitk
#                 slice_img = li_slices_image_unique[slice_num]   #format sitk
#                 slice_sain = mask_sain[:,:,slice_num_ref]
#                 #print(np.sum(sitk.GetArrayFromImage(slice_sain)))
#                 lsif = sitk.LabelStatisticsImageFilter()
#                 lsif.Execute(slice_img, slice_mask)
#                 boundingBox = np.array(lsif.GetBoundingBox(label))
#                 ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
#                 li_caled[image_num_local][slice_num_ref] = {"img": slice_img, "mask": slice_mask,"sain": slice_sain , "decals": (li_best_shift_x[image_num_local],  li_best_shift_y[image_num_local], li_best_shift_z[image_num_local]), "accept": False, "diff": difference[image_num_local], "time": time_inj[image_num_local]}
#                 li_caled[image_num_local][slice_num_ref]["accept"] = True #Comprendre qui est accepté
#                 if (sitk.GetArrayFromImage(slice_mask).sum() > 0) & (ndims >= minDims):
#                     try:
#                         featureVector = extractor.execute(slice_img, slice_mask)
#                         featureVector['slice_num'] = slice_num_ref #Tres important: on ne veut pas le slice_num mais le slice_num_ref
#                         df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
#                         df_idx['classe_name'] = classe_name
#                         df_idx['temps_inj'] = temps_inj
#                         df_idx['patient_num'] = patient_num
#                         df_idx = pd.DataFrame(df_idx).T
#                         al_dat_slice = pd.concat([al_dat_slice,df_idx])               
#                     except Exception as e:
#                         print("problem mask classic with slice", slice_num, "of image", imageName, "erreur", e)
#     if show_plots:
#         set_slice_num_ref_present = set([])
#         for radio in li_caled.values():
#             set_slice_num_ref_present = set(list(radio.keys())).union(set_slice_num_ref_present)
#         slice_num_ref_present = list(set_slice_num_ref_present)
#         for num_ref in slice_num_ref_present:
#             li_slices = []
#             for num_radio in range(len(li_caled)):
#                 if num_ref in list(li_caled[num_radio].keys()):
#                     li_slices.append(li_caled[num_radio][num_ref])
#             plt.figure(figsize = (15,5))
#             for num_radio, dic_slice in enumerate(li_slices):
#                 mask_superpose_simple(dic_slice["img"], dic_slice["mask"], other_mask = dic_slice["sain"],num = num_radio + 1, max_num = len(li_slices), legend = f"patient ls num = {num} slice {num_ref},\n recalage = {dic_slice['decals']},\n accept = {dic_slice['accept']}, diff = {dic_slice['diff']}, time = {dic_slice['time']}")
#                 #print(num_radio, "num_slice", num_ref)
#             plt.subplots_adjust(wspace=2.5)
#             plt.show()


In [150]:
# if double:
#     al_dat_slice.to_csv('./liver_tumors_slices_LLB.csv')
#     al_dat_slice.to_excel('../multislice_excel_with_shape_2D_sain_2.xlsx', index = False)
# data_sain.to_csv('./liver_sain_go.csv')
# data_sain.to_excel('../liver_sain_go.xlsx', index = False)

Vieux

In [151]:
# for num, li_image_names in tqdm.tqdm(enumerate(ls_image_full_time)):
#     li_mask_names = [image_name.replace('_NAT','').replace('.nii','_masked.nii') for image_name in li_image_names]
    
    
#     classe_name = li_image_names[0].split('/')[-2].split('_')[1]
#     patient_num = li_image_names[0].split('/')[-1].split('_')[0]
    
#     li_images = [sitk.ReadImage(image_name) for image_name in li_image_names]
#     li_mask = [sitk.ReadImage(mask_name) !=  0 for mask_name in li_mask_names]
    
#     li_slices_masks, li_slices_images = equalize_slices(ls_image_full_time, dict_image_full_time,num = num, key = None, show = False)
#     li_decal = np.array(find_best_decal_all_times_short(li_slices_masks, fenetre_z, fenetre_x=fenetre_x, fenetre_y=fenetre_y, mode=mode))
#     nb_slices = len(li_slices_images[0])
#     print("decalage", li_decal)

#     for image_num_local, li_slices_image_unique in enumerate(li_slices_images):
#         slice_depart = abs(np.min(li_decal[image_num_local],0))
#         slice_fin = nb_slices - np.max(li_decal[image_num_local],0)
#         for slice_num_ref in range(slice_depart, slice_fin):
#             slice_num = slice_num_ref + li_decal[image_num_local]
#             li_slices_mask_unique = li_slices_masks[image_num_local]
#             slice_mask = li_slices_mask_unique[slice_num]   #format numpy
#             slice_img = li_slices_image_unique[slice_num]   #format sitk
#             slice_mask = sitk.GetImageFromArray(slice_mask) #passer en sitk
#             slice_mask.CopyInformation(slice_img)
            
#             lsif = sitk.LabelStatisticsImageFilter()
#             lsif.Execute(slice_img, slice_mask)
#             boundingBox = np.array(lsif.GetBoundingBox(label))
#             ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
#             if (sitk.GetArrayFromImage(slice_mask).sum() > 0) & (ndims >= minDims):
#                 slice_mask_sain = cercle_image_compos_connexe(slice_img,slice_mask) #on bosse au départ avec du sitk (pour toutes nos fonctions...)
#                 #On calcule le slice mask sain
#                 slice_mask_sain = sitk.GetImageFromArray(slice_mask_sain)
#                 slice_mask_sain.CopyInformation(slice_img)
                
#                 lsif = sitk.LabelStatisticsImageFilter()
#                 lsif.Execute(slice_img, slice_mask_sain)
#                 boundingBox = np.array(lsif.GetBoundingBox(label))
#                 ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
#                 imageName = li_image_names[image_num_local]
#                 if (sitk.GetArrayFromImage(slice_mask_sain).sum() > 0) & (ndims >= minDims):
#                     try:
#                         problem = False
#                         #print("for extrac",ndims)
#                         feature_vector_sain = extractor_sain.execute(slice_img, slice_mask_sain)
#                         #print("extracted")
#                         featureVector = extractor.execute(slice_img, slice_mask)
#                         featureVector['slice_num'] = slice_num_ref #Tres important: on ne veut pas le slice_num mais le slice_num_ref
#                         df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
#                         df_idx_sain = pd.DataFrame(dict([ ("sain_" + k,pd.Series(v)) for k,v in feature_vector_sain.items() ])).iloc[0]
#                         df_idx = pd.concat([df_idx,df_idx_sain])
#                         df_idx['classe_name'] = classe_name
#                         temps_inj = imageName.split('_')[-1].split('.')[0]
#                         df_idx['temps_inj'] = temps_inj
#                         df_idx['patient_num'] = patient_num
#                         # al_dat_slice = al_dat_slice.append(df_idx)
#                         df_idx = pd.DataFrame(df_idx).T
#                         al_dat_slice = pd.concat([df_idx,al_dat_slice])
#                     except Exception as e:
#                         print(f'erreur pour slice {slice_num} pour image {imageName}')
#                         print(e)
#                         problem = True
#                 else:
#                     problem = True
#                     print(f'slice {slice_num} non valide sur zone saine pour image {imageName}')
#                 if problem:
#                     feature_vector_sain = extractor_sain.execute(slice_img, slice_mask)
#                     featureVector = extractor.execute(slice_img, slice_mask)
#                     featureVector['slice_num'] = slice_num
#                     df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
#                     df_idx_sain = pd.DataFrame(dict([ ("sain_" + k,pd.Series(v)) for k,v in feature_vector_sain.items() ])).iloc[0]
#                     df_idx_sain[:] = np.nan
#                     df_idx = pd.concat([df_idx,df_idx_sain])
#                     df_idx['classe_name'] = classe_name
#                     df_idx['temps_inj'] = temps_inj
#                     df_idx['patient_num'] = patient_num
#                     # al_dat_slice = al_dat_slice.append(df_idx)
#                     df_idx = pd.DataFrame(df_idx).T
#                     al_dat_slice = pd.concat([df_idx,al_dat_slice])
                # #print(list(df_idx.columns.values))
                # #print(list(featureVector.keys()))
        

In [152]:
# print(extractor.enabledFeatures)
# print(extractor_sain.enabledFeatures)